In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
import cv2 as cv
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
(x_train_pre, y_train_pre) , (x_test_pre,y_test_pre) = keras.datasets.cifar10.load_data()

x_full = np.concatenate([x_train_pre, x_test_pre], axis=0)
y_full = np.concatenate([y_train_pre, y_test_pre], axis=0)

x_train, x_temp, y_train, y_temp = train_test_split(x_full, y_full, test_size=0.30,random_state=41)

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.50,random_state=35)



In [3]:
things = ['airplane','automobile','bird','cat','deer','dog','frog',
          'horse','ship','truck']

In [16]:
#adds the layers that are processed in order for data augmentaton to occur 

data_augmentation = tf.keras.Sequential([
    #tf.keras.layers.RandomCrop(32, 32),
    layers.Rescaling(1./255), 
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),

])

#sets the build for the augmentation layer so it fits with the model 
#properly 


In [17]:
def residual_block(x,filters,reg,dropout_rate,downsample=False, expansion=4):

    #what this does is save the input value which is kinda like the important thing abt residual
    shortcut_x = x 
    strides = 2 if downsample else 1

    x = layers.Conv2D(filters, (1,1), padding='same', strides=1,
                      kernel_initializer='he_normal', kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(filters, (3,3), padding='same', strides=strides,
                      kernel_initializer='he_normal', kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(filters * expansion, (1,1), padding='same', strides=1,
                      kernel_initializer='he_normal', kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)

    if downsample or shortcut_x.shape[-1] != filters * expansion:
        shortcut_x = layers.Conv2D(filters * expansion, (1,1), padding='same', strides=strides,
                                   kernel_initializer='he_normal', kernel_regularizer=reg)(shortcut_x)
        shortcut_x = layers.BatchNormalization()(shortcut_x)
        
    x = layers.Add()([x, shortcut_x])
    x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)
    
    return x


In [ ]:

regularizer = regularizers.L2(1e-4)

inputs = keras.Input(shape=(32,32,3))

x = data_augmentation(inputs)

x = layers.Conv2D(64, kernel_size=(7,7), strides=2, kernel_initializer='he_normal', 
                  kernel_regularizer=regularizer, padding='same')(x)    
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = residual_block(x, 64, regularizer, dropout_rate=0.1, downsample=True)
x = residual_block(x, 64, regularizer, dropout_rate=0.1)

x = residual_block(x, 128, regularizer, dropout_rate=0.15, downsample=True)
x = residual_block(x, 128, regularizer, dropout_rate=0.15)

x = residual_block(x, 256, regularizer, dropout_rate=0.2, downsample=True)
x = residual_block(x, 256, regularizer, dropout_rate=0.2)

x = layers.GlobalAveragePooling2D()(x)

x= layers.Dense(256, activation='relu', kernel_initializer='he_normal',kernel_regularizer=regularizer)(x)
x= layers.BatchNormalization()(x)

x = layers.Dropout(0.5)(x)

output= layers.Dense(10, activation='softmax',
                          kernel_initializer='he_normal',kernel_regularizer=regularizer)(x)





In [30]:
resmodel = keras.Model(inputs,output)

decay_steps = 657 * 25
initial_learning_rate = 0.0008
alpha = .001
lr_decayed_fn = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps,alpha)

optimizer = tf.keras.optimizers.AdamW(lr_decayed_fn)

resmodel.compile(optimizer=optimizer, 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
    metrics=['accuracy'])
    

In [ ]:
steps = np.arange(0, decay_steps)
lrs = [lr_decayed_fn(step) for step in steps]

plt.figure(figsize=(10, 6))
plt.plot(steps, lrs)
plt.title('Cosine Decay Learning Rate Schedule')
plt.xlabel('Training Step')
plt.ylabel('Learning Rate')
plt.yscale('log')
plt.vlines(x=25*657,ymin=0,ymax=10e-3)
plt.grid(True)
plt.show()

In [ ]:
resmodel.summary()

In [ ]:
history = resmodel.fit(       
    x_train, y_train, batch_size=64,
    validation_data=(x_val,y_val),
    epochs=25,
    callbacks=[
    keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,  
    restore_best_weights=True,
    verbose=1),
    ])

In [ ]:
resmodel.fit(x_test,y_test)